In [8]:
# Cell 1: Imports and parameters
import os, glob
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# -----------------------------
# User parameters
# -----------------------------
main_path = '/Volumes/data_backup/mercury/extreme/CPN_Base/05/particles/'
species = np.array(['H+', 'He++'])
sim_ppc = [24, 11]
sim_den = [38.0e6, 1.0e6]
sim_vel = [400.e3, 400.e3]
sim_dx = sim_dy = sim_dz = 75.e3
sim_robs = 2440.e3
select_R = 2480.e3
dphi = 2.
dtheta = 2.


In [9]:
# Cell 2: Function to combine NetCDF files
def combine_netcdfs(main_path, select_R):
    files = sorted(glob.glob(os.path.join(main_path, "*.nc")))
    print(f"Found {len(files)} NetCDF files.")

    # Initialize empty lists
    prx = pry = prz = pvx = pvy = pvz = psid = np.array([])

    for f in files:
        print("Reading:", f)
        with Dataset(f, 'r') as nc:
            prx = np.append(prx, nc.variables['rx'][:])
            pry = np.append(pry, nc.variables['ry'][:])
            prz = np.append(prz, nc.variables['rz'][:])
            pvx = np.append(pvx, nc.variables['vx'][:])
            pvy = np.append(pvy, nc.variables['vy'][:])
            pvz = np.append(pvz, nc.variables['vz'][:])
            psid = np.append(psid, nc.variables['sid'][:])

    # Filter particles within selection radius
    idx = np.where(prx**2 + pry**2 + prz**2 <= select_R**2)[0]
    prx, pry, prz = prx[idx], pry[idx], prz[idx]
    pvx, pvy, pvz = pvx[idx], pvy[idx], pvz[idx]
    psid = psid[idx]

    print("Total number of particles after selection:", prx.size)

    return prx, pry, prz, pvx, pvy, pvz, psid


In [ ]:
# Cell 5: Load/combine data
rx, ry, rz, vx, vy, vz, sid = combine_netcdfs(main_path, select_R)


Found 4 NetCDF files.
Reading: /Volumes/data_backup/mercury/extreme/CPN_Base/05/particles/Subset_CPN_Base_112000_full_domain.nc


In [ ]:
# Cell 3: Function to calculate moments at surface
def calc_moments_at_surface(rx, ry, rz, vx, vy, vz, sid, specie_id):
    from math import pi

    # Select particles by species
    idx = np.where((sid == specie_id) & (rx**2 + ry**2 + rz**2 <= select_R**2))[0]
    prx = rx[idx]; pry = ry[idx]; prz = rz[idx]
    pvx = vx[idx]; pvy = vy[idx]; pvz = vz[idx]
    tnp = prx.size

    print(f"Total number of particles for species {species[specie_id]}: {tnp}")

    # Initialize bins
    nx = int(360/dphi) + 2
    ny = int(180/dtheta) + 2
    cnts = np.zeros((nx, ny))
    velx = np.zeros((nx, ny))
    vely = np.zeros((nx, ny))
    velz = np.zeros((nx, ny))
    velr = np.zeros((nx, ny))
    den  = np.zeros((nx-2, ny-2))

    # Bin particles
    for i in range(tnp):
        rmag = np.sqrt(prx[i]**2 + pry[i]**2 + prz[i]**2)
        theta = np.arccos(prz[i]/rmag) * 180/pi
        phi   = np.arctan2(pry[i], prx[i]) * 180/pi
        if phi < 0: phi += 360

        xi = int(np.floor(phi/dphi - 0.5)) + 1
        yi = int(np.floor(theta/dtheta - 0.5)) + 1
        u = xi + 1; v = yi + 1
        x = xi + 0.5 - (phi/dphi)
        y = yi + 0.5 - (theta/dtheta)

        r_dot_v = abs(prx[i]*pvx[i] + pry[i]*pvy[i] + prz[i]*pvz[i])/rmag
        c_list = [(xi, yi, (1-x)*(1-y)), (u, yi, x*(1-y)),
                  (xi, v, (1-x)*y), (u, v, x*y)]
        for ii, jj, c in c_list:
            cnts[ii, jj]  += c
            velx[ii, jj] += c*pvx[i]
            vely[ii, jj] += c*pvy[i]
            velz[ii, jj] += c*pvz[i]
            velr[ii, jj] += c*r_dot_v

    # Correct guard cells
    cnts[1,:] += cnts[-1,:]; velx[1,:] += velx[-1,:]; vely[1,:] += vely[-1,:]
    velz[1,:] += velz[-1,:]; velr[1,:] += velr[-1,:]

    cnts = cnts[1:-1, 1:-1]; velx = velx[1:-1, 1:-1]
    vely = vely[1:-1, 1:-1]; velz = velz[1:-1, 1:-1]; velr = velr[1:-1, 1:-1]

    # Density
    weight = (sim_dx*sim_dy*sim_dz)/sim_ppc[specie_id]
    dr = select_R - sim_robs
    dv = (select_R**2)*dr*(dphi*pi/180)*(dtheta*pi/180)
    for i in range(cnts.shape[0]):
        den[i,:] = cnts[i,:] * sim_den[specie_id] * weight / (dv*np.sin((i+0.5)*dtheta*pi/180))

    # Average velocities
    mask = cnts>0
    velx[mask] /= cnts[mask]; vely[mask] /= cnts[mask]
    velz[mask] /= cnts[mask]; velr[mask] /= cnts[mask]

    vmag = np.sqrt(velx**2 + vely**2 + velz**2)
    flxr = velr * den

    return cnts, den, velr, flxr


In [ ]:
# Cell 6: Compute moments for a species (run once)
species_id = 0  # 0 = H+, 1 = He++
cnts, den, velr, flxr = calc_moments_at_surface(rx, ry, rz, vx, vy, vz, sid, species_id)


In [ ]:
# Cell 4: Function to plot moments (with log10 fix and Aitoff full globe)
def plot_moments_at_surface(cnts, den, velr, flxr, specie_id):
    interp = 'bilinear'

    # Avoid log10(0)
    log_cnts = np.where(cnts>0, np.log10(cnts), np.nan)
    log_den  = np.where(den>0, np.log10(den), np.nan)
    log_velr = np.where(velr>0, np.log10(velr), np.nan)
    log_flxr = np.where(flxr>0, np.log10(flxr), np.nan)

    fig = plt.figure(species[specie_id], figsize=(14,9))

    ax = plt.subplot(221, projection=ccrs.Aitoff())
    ax.gridlines(color='black', linestyle='dotted')
    im = ax.imshow(np.flipud(log_cnts.T), origin="upper", interpolation=interp,
                   extent=(-180,180,-90,90), transform=ccrs.PlateCarree())
    plt.colorbar(im, extend='neither', ax=ax).set_label("# particles")

    ax = plt.subplot(222, projection=ccrs.Aitoff())
    ax.gridlines(color='black', linestyle='dotted')
    im = ax.imshow(np.flipud(log_den.T), origin="upper", interpolation=interp,
                   extent=(-180,180,-90,90), transform=ccrs.PlateCarree())
    plt.colorbar(im, extend='min', ax=ax).set_label("Density")

    ax = plt.subplot(223, projection=ccrs.Aitoff())
    ax.gridlines(color='black', linestyle='dotted')
    im = ax.imshow(np.flipud(log_velr.T), origin="upper", interpolation=interp,
                   extent=(-180,180,-90,90), transform=ccrs.PlateCarree())
    plt.colorbar(im, extend='min', ax=ax).set_label("Radial velocity")

    ax = plt.subplot(224, projection=ccrs.Aitoff())
    ax.gridlines(color='black', linestyle='dotted')
    im = ax.imshow(np.flipud(log_flxr.T), origin="upper", interpolation=interp,
                   extent=(-180,180,-90,90), transform=ccrs.PlateCarree())
    plt.colorbar(im, extend='min', ax=ax).set_label("Flux")

    plt.show()


In [ ]:
# Cell 7: Plot moments
plot_moments_at_surface(cnts, den, velr, flxr, species_id)
